In [1]:
###############################################################################
# Python code to determine Jost function values                               #
# from Legendre-mesh R-matrix calculations                                    #
#                                                                             #
# Paul Vaandrager, 9 May 2022                                                 #
# There will be a lot of unnecessary comments because I am learning Python    #
###############################################################################
# ----------------------------------------------------------------------------
# Stuff to import before we begin: 
# ----------------------------------------------------------------------------
import quadpy # this is for complex integrals
import numpy as np # most important - use rahter than math and cmath!
from scipy.special import legendre # getting legendre polynomial coefficients
from scipy.special import eval_legendre # evaluating legendre polynomial at a value
from numpy.polynomial.legendre import leggauss # getting the legendre polynomial zeros
from mpmath import *  # Coulomb functions, F(l,eta,z) and G(l,eta,z)

In [2]:
# ============================================================================
# Values of system
# ----------------------------------------------------------------------------
hbar = 1.0
mu = 1.0    # reduced mass of 2 body system
l = 0       # angular momentum quantum number
E = 0.1     # energy to determine Jost, S-matrix, phase, etc
N = 40      # Number of legendre functions
a = 5.0     # channel radius
k = np.sqrt(2.0*mu*E/hbar**2.0)
etav = 0.0 #--------------------------------------- Sommerfeld parameter - this is not generally 0!
small = 0.1E-5 # this is the value for r->0
# -----------------------------------------------------------------------------
# General parameters
# -----------------------------------------------------------------------------
zero = 0.0
I = np.identity(N)
i = j # I just prefer i
# -----------------------------------------------------------------------------
# Potential:
# -----------------------------------------------------------------------------
# This is the single-channel Bargmann Potential, with parameters from [1]
b = 2.0
c = -1.0
beta = (b-c)/(b+c)
def V(r):
    return -4.0*b*b*beta*((np.exp(-2.0*b*r))/(1.0+beta*np.exp(-2.0*b*r))**2.0)
# -----------------------------------------------------------------------------
# Gauss-Legendre (default interval is [-1, 1])
# -----------------------------------------------------------------------------
x, w = leggauss(N)
#print('x=',x)
x = (x + 1.0) / 2.0  # change to interval [0,1]
w = w / 2.0  # change to interval [0,1]
#print('x=',x)
#print('w=',w)
# -----------------------------------------------------------------------------

In [3]:
# -----------------------------------------------------------------------------
# The Matrix C
# -----------------------------------------------------------------------------
T0plusL = np.zeros((N, N))
Tl = np.zeros((N, N))
VM = np.zeros((N, N))
C = np.zeros((N, N))
for n0 in range(N):
    for n1 in range(N):
        if n0==n1: # diagonal elements
            T0plusL[n0,n1]=1.0/(6.0*a*a*x[n0]*(1.0-x[n0]))*(4.0*N*(N+1.0)+3.0+(1-6.0*x[n0])/(x[n0]*(1.0-x[n0]))) # T at l=0
            VM[n0,n1] = V(x[n0] * a) # potential matrix
            Tl[n0,n1] = 0.5*(l*(l+1.0)/(a*a*x[n0]*x[n0])) # term of T dependent on l
            C[n0,n1]=T0plusL[n0,n1]+Tl[n0,n1]+VM[n0,n1]
        else: # off-diagonal elements
            T0plusL[n0,n1]=((-1.0)**((n0+1.0)+(n1+1.0)))/(2.0*a*a*(x[n0]*x[n1]*(1.0-x[n0])*(1-x[n1]))**(0.5))*(N*(N+1.0)
                                                                                                +
                         1.0+(x[n0]+x[n1]-2.0*x[n0]*x[n1])/((x[n0]-x[n1])**2.0)-(1.0/(1.0-x[n0]))-(1.0/(1.0-x[n1])))
            C[n0,n1]=T0plusL[n0,n1]
#print('T+L=',TplusL)
#print('VM=',VM)
CEI = C-E*I # the final matrix C-EI
CEIinv = np.linalg.inv(CEI) # here we invert the C-EI matrix. This is not necessary and needs to be optimized.
#print('C-EI=',CEI)
#print('(C-EI)inv=',CEIinv)
# -----------------------------------------------------------------------------
# -----------------------------------------------------------------------------
# The function f(a) (specifically at r=a)
# -----------------------------------------------------------------------------
fa = np.zeros(N) # evaluate PN(1)
for n0 in range(N):
    #print('n0=',n0)
    fa[n0]=((-1.0)**(N-(n0+1.0)))*(1.0/np.sqrt(a*x[n0]*(1.0-x[n0])))
#print('f(a)=',fa)
# -----------------------------------------------------------------------------
# The R-Matrix
# -----------------------------------------------------------------------------
sum = 0.0
for n0 in range(N):
    for n1 in range(N):
        sum = sum + fa[n0]*CEIinv[n0,n1]*fa[n1]
R = ((hbar**2.0)/(2.0*mu*a))*sum # this is the computational R-matrix
# -----------------------------------------------------------------------------
# S-matrix and phase-shift from R-matrix
# -----------------------------------------------------------------------------
F=coulombf(l, etav, k*a)
#print('F(ka)',F)
G=coulombg(l, etav, k*a)
Fp1=coulombf(l+1, etav, k*a) # This is F of l+1, required to find F' from recurrence relation
Gp1=coulombg(l+1, etav, k*a) # This is G of l+1, required to find G' from recurrence relation
Rclp1=np.sqrt(1.0+etav**2.0/(l+1.0)**2.0) # This is a function Rcl of l+1, required to find F' from recurrence relation
Sclp1=(l+1)/(k*a)+etav/(l+1) # This is a function Scl of l+1, required to find G' from recurrence relation
Fd=Sclp1*F-Rclp1*Fp1 # This is F'
Gd=Sclp1*G-Rclp1*Gp1 # This is G'
I=G-i*F
O=G+i*F
Id=Gd-i*Fd
Od=Gd+i*Fd
S = (I-k*a*R*Id)/(O-k*a*R*Od)
Sigma = (log(S)/(i*2.0))*180.0/np.pi+180
# -----------------------------------------------------------------------------

In [4]:
# -----------------------------------------------------------------------------
# The wave function at any r from the computational R-matrix
# -----------------------------------------------------------------------------
def Pn(r): #SUBROUTINE - The shifted Legendre polynomials at any r
    return eval_legendre(N, 2*(r/a)-1.0)
def uR(r): # SUBROUTINE - call for specific "r"
    fn = 0
    sum = 0
    for n0 in range(N):
        fn=((-1.0)**(N-(n0+1.0)))*(1/(np.sqrt(a)))*(np.sqrt((1-x[n0])/x[n0]))*r*Pn(r)/(r-a*x[n0])
        for n1 in range(N):
            sum = sum + fn*CEIinv[n0,n1]*fa[n1]
    uRval = ((hbar**2.0)/(2.0*mu))*(Id-S*Od)*sum
    return uRval # this just makes it a numpy format so we can use quadpy
# -----------------------------------------------------------------------------
# Normalization constant A or N for l=0 (Jeremy's suggestion)
# -----------------------------------------------------------------------------
sum = 0
limf = np.zeros((N))
for n0 in range(N):
    limf[n0] = ((-1.0)**(n0+2.0))*np.sqrt((1.0-x[n0])/((a*x[n0])**(2.0*l+3.0))) # Limit f(r)/r function for l=0 (Jeremy's suggestion)
    for n1 in range(N):
        sum = sum + limf[n0]*CEIinv[n0,n1]*fa[n1]
AN = (((hbar**2.0)/(2.0*mu))*(Id-S*Od)*sum)**(-1.0)
jost=-2.0*i*AN
# -----------------------------------------------------------------------------

In [5]:
# -----------------------------------------------------------------------------
# Very rough approximation of normalization factor at r -> 0
# to find u(r) which is regular at the origin
# -----------------------------------------------------------------------------
F1=coulombf(l, etav, k*small) # this is the regular Coulomb function at small r
#print(F1)
AN1 =  2.0*F1/(uR(small))  # this is the normalization factor (very rough)
jost1=-2.0*i*AN1     # this is the jost function from the rough normalization factor
# -----------------------------------------------------------------------------

In [6]:
#------------------------------------------------------------------------------
# Exact Jost, S-matrix and R-matrix for Bargmann Potential at l=0
# -----------------------------------------------------------------------------
exactfin=(k+i*(-1.0))/(k+i*2.0)
exactfout=np.conj(exactfin)
exactS=(exactfout)/(exactfin)
exactR=(I-exactS*O)/(k*a*(Id-exactS*Od))
exactSigma=(log(exactS)/(i*2.0))*180.0/np.pi+180
# -----------------------------------------------------------------------------

In [7]:
# -----------------------------------------------------------------------------
# Print Results
# -----------------------------------------------------------------------------
print('----------------------------------------------------------------------------------------')
print('E=',E)
print('N=',N)
print('k=',k)
print('l=',l)
print('----------------------------------------------------------------------------------------')
if l == 0:
    print('R             =',R)
    print('Exact R (l=0) =',exactR.real)
else:
    print('R             =',R)
print('----------------------------------------------------------------------------------------')
if l == 0:
    print('S             =',S)
    print('Exact S (l=0) =',exactS)
else:
    print('S             =',S)
print('----------------------------------------------------------------------------------------')
if l == 0:
    print('phase             =',format(complex(''.join(format(Sigma).split())),'1.9E'))
    print('Exact phase (l=0) =',format(complex(''.join(format(exactSigma).split())),'1.9E'))
else:
    print('phase             =',format(complex(''.join(format(Sigma).split())),'1.9E'))
print('----------------------------------------------------------------------------------------')

if l == 0:
    print('Normalization coefficient from Legendre polynomial for l=0=',AN)
    print('Normalization coefficient from F(kr) limit as r->0        =', AN1)
    print('with r->0 approximated by r =',small)
else:
    print('Normalization coefficient from F(kr) limit as r->0        =', AN1)
print('----------------------------------------------------------------------------------------')
if l == 0:
    # All this "format" stuff is just to get it to print something that looks exactly like Fortran output
    print('Jost from Legendre polynomial (l=0)  =',format(complex(''.join(format(jost).split())),'1.9E'))
    print('Exact Jost (l=0)                     =',format(complex(''.join(format(exactfin).split())),'1.9E'))
    print('Jost from F(kr) limit as r->0        =',format(complex(''.join(format(jost1).split())),'1.9E'))
else:
    print('Jost from F(kr) limit as r->0 =',format(complex(''.join(format(jost1).split())),'1.9E'))
print('----------------------------------------------------------------------------------------')

----------------------------------------------------------------------------------------
E= 0.1
N= 40
k= 0.4472135954999579
l= 0
----------------------------------------------------------------------------------------
R             = -18.06610827845147
Exact R (l=0) = -18.066029918439
----------------------------------------------------------------------------------------
S             = (0.285714080052149 - 0.958314908816488j)
Exact S (l=0) = (0.285714285714286 - 0.95831484749991j)
----------------------------------------------------------------------------------------
phase             = 1.433007687E+02-1.447770406E-15j
Exact phase (l=0) = 1.433007748E+02-6.940877802E-16j
----------------------------------------------------------------------------------------
Normalization coefficient from Legendre polynomial for l=0= (0.159719177366721 - 0.214285714753565j)
Normalization coefficient from F(kr) limit as r->0        = (0.142857175158517 - 0.191662969915199j)
with r->0 approximated by 